In [1]:
#import required packages
import pandas as pd
import numpy as np

In [2]:
#read the data
energy_pd = pd.read_csv("data/Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_-_FY2020.csv")

#take a look
print(energy_pd.describe())

           Postcode  Current fulltime  Job created   Job retain  \
count    450.000000        446.000000    11.000000    14.000000   
mean   11024.124444        124.186099    13.545455   297.500000   
std      405.126704        392.750810    16.409532   778.404655   
min    10001.000000          0.000000     2.000000     4.000000   
25%    11101.000000         11.000000     4.000000    15.500000   
50%    11205.000000         28.000000    10.000000    42.000000   
75%    11232.000000         76.000000    13.500000   160.750000   
max    11693.000000       3878.000000    60.000000  2964.000000   

       Total Savings  Savings from beginning receiving benefits    Latitude  \
count   4.500000e+02                               4.500000e+02  409.000000   
mean    3.943778e+04                               4.234601e+05   40.722293   
std     1.345949e+05                               1.936066e+06    0.057503   
min    -1.099101e+05                               0.000000e+00   40.555080   
2

In [3]:
##how many companies are included?

#extract all the value of the column with company names
company_names = energy_pd["Company Name"]

#extract the unique cases using set
unique_company_names = set(company_names)

#print results
print(f"There are {len(unique_company_names)} different companies")

There are 439 different companies


In [4]:
##total number of jobs created in Queens

#which rows belong to Queens
boolean_queens = energy_pd["City"] == "Queens"

#filter by Queens
energy_pd_queens = energy_pd[boolean_queens]

#select the column of jobs created
jobs_created_queens = energy_pd_queens["Job created"].sum()

#print
print(f"The number of jobs created in Queens was {jobs_created_queens}")

The number of jobs created in Queens was 90.0


In [5]:
##unique email domains

#select the column with emails
company_emails = energy_pd["company email"]

#remove NAs
company_emails_no_na = company_emails[company_emails.notna()]

#for each email, split by "@" and the select the domain
list_domains = [email.split("@")[-1] for email in company_emails_no_na]

#extract the number of unique domains using set
n_unique_domains = len(set(list_domains))

#print
print(f"The number of unique email domains is {n_unique_domains}")

The number of unique email domains is 361


In [6]:
##work with NTAs having at least 5 businesses

#extract NTAs
ntas = energy_pd["NTA"]

#remove nas
ntas = ntas[ntas.notna()]

#select unique cases
unique_ntas = list(set(ntas))

#create df to save results
results = pd.DataFrame(unique_ntas)
results = results.rename(columns={0:"NTA"})
results["Average Savings"] = ""
results["Total Jobs"] = ""

#for each nta calculate the average savings and jobs creates
for nta in unique_ntas:
    
    #select the rows for nta
    selected_subset = energy_pd[energy_pd["NTA"] == nta]
    
    #get the company names
    selected_subset_companies = list(set(selected_subset["Company Name"]))
    
    #if the number of different businesses is equal or larger than 5
    if len(selected_subset_companies) >= 5:
        
        #calculate the average of savings and jobs within the selected nta
        avg_savings = np.nanmean(selected_subset["Total Savings"])
        total_jobs = np.nansum(selected_subset["Job created"])
            #use np.nan just in case there are nans
        
        #save the results for the corresponding NTA
        results.loc[results["NTA"] == nta, "Average Savings"] = avg_savings
        results.loc[results["NTA"] == nta, "Total Jobs"] = total_jobs

        
#convert empty cells to NA
results = results.replace('', np.nan)

#remove NAs
results = results.dropna()

#see the results
results

,NTA,Average Savings,Total Jobs
0,Greenpoint,9753.649286,0.0
6,College Point,17159.372857,2.0
8,park-cemetery-etc-Brooklyn,14327.419333,0.0
9,Manhattanville,2760.888889,0.0
10,East Williamsburg,13990.626364,0.0
11,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,59950.340000,0.0
20,Crown Heights North,4036.298889,0.0
21,Maspeth,6658.283333,10.0
28,Hunts Point,39246.016154,12.0
33,SoHo-TriBeCa-Civic Center-Little Italy,37214.024000,0.0


In [7]:
##save the previous results as csv
results.to_csv("results.csv", sep=",")